# utility functions

In [1]:
def make_undirected(G):
    """Makes directed graph undirected by adding reverse edges."""
    V,E = G
    reverse_edges = []
    for (u,v) in E:
        if (v,u) not in E and (v,u) not in reverse_edges:
            reverse_edges.append((v,u))
    return (V, E+reverse_edges)

# Path

In [2]:
import collections
def path(G,s,t):
    """Check if there is a path from node s to node t in graph G."""
    V,E = G
    visited = []
    queue = collections.deque([s])
    while queue: # |\label{bfs:line-outer-loop}|
        node = queue.popleft()
        if node == t:
            return True
        if node not in visited:
            visited.append(node)
            node_neighbors = [ v for (u,v) in E if u==node ] # |\label{bfs:line-find-neighbors}|
            for neighbor in node_neighbors: # |\label{bfs:line-inner-loop}|
                if neighbor not in visited:
                    queue.append(neighbor)
    return False

In [3]:
V = [1,2,3,4,5,6,7,8]
E = [ (1,2), (2,3), (3,4), (4,5), (6,7), (7,8), (8,6) ]
G = (V,E)
print "path from {} to {}? {}".format(4, 1, path(G, 4, 1))
print "path from {} to {}? {}".format(1, 4, path(G, 1, 4))

G = make_undirected(G)
print "path from {} to {}? {}".format(4, 1, path(G, 4, 1))
print "path from {} to {}? {}".format(1, 4, path(G, 1, 4))
print "path from {} to {}? {}".format(4, 7, path(G, 4, 7))

path from 4 to 1? False
path from 1 to 4? True
path from 4 to 1? True
path from 1 to 4? True
path from 4 to 7? False


# relatively prime

In [4]:
def gcd(x,y):
    """Euclid's algorithm for greatest common divisor."""
    while y>0:
        x = x % y
        x,y = y,x
    return x

def rel_prime(x,y):
    return gcd(x,y) == 1

In [5]:
print gcd(24,60), rel_prime(24,60)
print gcd(25,63), rel_prime(25,63)

12 False
1 True


# Connected

In [6]:
import itertools
def connected(G):
    """Check if G is connected."""
    V,E = G
    for (s,t) in itertools.combinations(V,2):
        if not path(G,s,t):
            return False
    return True

In [7]:
V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,6) ]
G = (V,E)
G = make_undirected(G)
print connected(G)

V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,6), (2,5) ]
G = (V,E)
G = make_undirected(G)
print connected(G)

False
True


# Eulerian path

In [8]:
def degree(node, E):
    """Return degree of node."""
    return sum(1 for (u,v) in E if u==node)

def eulerian_path(G):
    """Check if G has an Eulerian path."""
    V,E = G
    num_deg_odd = 0
    V_pos = [] # nodes with positive degree
    for u in V:
        deg = degree(u, E)
        if deg % 2 == 1:
            num_deg_odd += 1
        if deg > 0:
            V_pos.append(u)
    if num_deg_odd not in [0,2]:
        return False
    G_pos = (V_pos,E)
    return connected(G_pos)

In [9]:
V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,6) ]
G = (V,E)
G = make_undirected(G)
print eulerian_path(G)

V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,1) ]
G = (V,E)
G = make_undirected(G)
print eulerian_path(G)

False
True


# Hamiltonian path verifier

In [10]:
def ham_path_verify(G, p):
    """Verify that p is a Hamiltonian path in G."""
    V,E = G
    # verify each pair of adjacent nodes in p shares an edge
    for i in range(len(p) - 1):
        if (p[i],p[i+1]) not in E:
            return False
    # verify each node in V appears in p
    if set(p) != set(V):
        return False
    # verify each node in V appears exactly once in p
    if len(set(p)) != len(p):
        return False
    return True

In [11]:
V = [1,2,3,4,5]
E = [ (1,2), (2,4), (4,3), (3,5), (3,1)]
G = (V,E)

p_bad = [1,2,3,4,5]
print ham_path_verify(G, p_bad)

p_bad2 = [1,2,4,3,1,2,4,3,5]
print ham_path_verify(G, p_bad2)

p_good = [1,2,4,3,5]
print ham_path_verify(G, p_good)

False
False
True


# composite verifier

In [12]:
def composite_verify(n,p):
    """Verify that p is a nontrivial divisor of n."""
    if not 1 < p < n:
        return False
    return n % p == 0 

In [13]:
print composite_verify(15, 3)
for p in range(1,20):
    print composite_verify(17, p), 

True
False False False False False False False False False False False False False False False False False False False


# NTM

In [14]:
import collections

def M(x):
    init_config = ("q0", 0, x+"_")
    def delta(state, symbol):
        return [ (state, symbol, 1), ("q0", "0", -1) ]
    visited = []
    queue = collections.deque([init_config])
    while queue:
        config = queue.popleft()
        if config.state == "qA":
            return True
        if config not in visited:
            visited.append(config)
            # get all possible next configurations; see below
            neighbors = next_configs(delta, config)
            for neighbor in neighbors:
                if neighbor not in visited:
                    queue.append(neighbor)
    return False

def next_configs(delta, config):
    """Apply delta to configuration (state, pos, content)
    to get list of possible next configurations."""
    state, pos, content = config # give names to each part of triple
    configs = []
    symbol = content[pos]
    for (new_state, new_symbol, move) in delta(state, symbol):
        new_content = content[:pos] + new_symbol + content[pos+1:]
        new_pos = max(0, pos + move)
        if new_pos >= len(new_content):
            new_content += "-"
        config = (new_state, new_pos, new_content)
        configs.append(config)
    return configs

#TODO: make up a more interesting NTM

In [15]:
import random

def N1(G):
    """NTM (nondeterminism implemented via random) deciding HamPath."""
    V,E = G
    # guess a path p that visits each node exactly once
    p = list(V)
    random.shuffle(p)
    # verify each pair of adjacent nodes in pi shares an edge
    for i in range(len(p) - 1):
        if (p[i],p[i+1]) not in E:
            return False
    print p, 
    return True

V = [1,2,3,4,5]
E = [ (1,2), (2,4), (4,3), (3,5), (3,1)]
G = (V,E)
for _ in range(100):
    print N1(G), 

False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False [1, 2, 4, 3, 5] True False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False


# Hamiltonian path exponential-time deterministic algorithm

In [16]:
import itertools

def ham_path_brute_force(G):
    """Exponential-time algorithm for finding Hamiltonian paths,
    which calls the verifier on all potential witnesses."""
    V,E = G
    for p in itertools.permutations(V):
        if ham_path_verify(G, p):
            return True
    return False

In [17]:
V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,6) ]
G = (V,E)
print ham_path_brute_force(G)

V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,5), (5,6) ]
G = (V,E)
print ham_path_brute_force(G)

False
True


# clique verifier

In [18]:
import itertools
def clique_v(G, k, C):
    """Verify C is a k-clique in G."""
    V,E = G
    # verify C is the correct size
    if len(C) != k:
        return False
    # verify each pair of nodes in C shares an edge
    for (u,v) in itertools.combinations(C, 2):
        if (u,v) not in E:
            return False
    return True

In [19]:
V = [1,2,3,4,5,6]
E = [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4), (4,5), (5,6), (4,6)]
G = (V,E)
G = make_undirected(G)

C = [1,2,3,4]
k = len(C)
print '{} is a {}-clique in G? {}'.format(C, k, clique_v(G, k, C))

C = [3,4,5]
print '{} is a {}-clique in G? {}'.format(C, k, clique_v(G, k, C))

[1, 2, 3, 4] is a 4-clique in G? True
[3, 4, 5] is a 4-clique in G? False


# subset sum verifier

In [20]:
import itertools
def subset_sum_v(C, t, S):
    """Verifies that S is a subcollection of C summing to t."""
    if sum(S) != t: # check sum
        return False
    C = list(C) # make copy that we can change
    for n in S: # ensure S is a subcollection of C
        if n not in C:
            return False
        C.remove(n)
    return True

In [21]:
C = [1,2,3,4,5,6,7,8]
t = 20
S = [1,2,3,4,5,6]
print subset_sum_v(C, t, S)

C = [1,2,3,4,5,6,7,8]
t = 20
S = [8, 7, 5]
print subset_sum_v(C, t, S)

False
True
